In [4]:
!pip install -U langchain langchain-community chromadb plotly scikit-learn numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.24.1
    Uninstalling plotly-5.24.1:
      Successfully uninstalled plotly-5.24.1
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.6
    Uninstalling langchain-text-splitters-0.3.6:
      Successfully uninstalled langchain-text-splitters-0.3.6
  A

In [7]:
import os
import numpy as np
from sklearn.manifold import TSNE
import plotly.graph_objs as go
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from google.colab import userdata
from langchain_core.documents import Document

In [8]:
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"  # Lightweight & fast
embedding = HuggingFaceEmbeddings(model_name=embedding_model)
print("Embedding model loaded successfully!")

Embedding model loaded successfully!


In [3]:
db_name = "/content/chroma_db"

In [4]:
if os.path.exists(db_name):
    vectorstore = Chroma(persist_directory=db_name, embedding_function=embeddings)
    vectorstore.delete_collection()

In [9]:
chunks = [
    Document(page_content="Our products are top-notch.", metadata={"doc_type": "products"}),
    Document(page_content="Meet our amazing employees.", metadata={"doc_type": "employees"}),
    Document(page_content="The new contract terms are beneficial.", metadata={"doc_type": "contracts"}),
    Document(page_content="Our company is growing rapidly.", metadata={"doc_type": "company"}),
]


In [11]:
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embedding,
    persist_directory=db_name
)

print(f"Vectorstore created with {vectorstore._collection.count()} documents")

Vectorstore created with 4 documents


In [12]:
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions} dimensions")

The vectors have 384 dimensions


In [18]:
llm_pipeline = pipeline("text-generation", model="mosaicml/mpt-7b", device_map="auto",max_new_tokens=50)
llm = HuggingFacePipeline(pipeline=llm_pipeline)
print("Hugging Face LLM loaded successfully!")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)


<ipython-input-18-9790a05eb1ab>:4: LangChainDeprecationWarning:

The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.



ValidationError: 1 validation error for ChatOpenAI
  Value error, Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. [type=value_error, input_value={'model_name': 'gpt-3.5-t...ne, 'http_client': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [ ]:
query = "What are the benefits of our new contract?"
response = qa_chain.invoke(query)
print("Response from QA Chain:", response)